# Extract contours from GeoJSON

In [ ]:
jsonfile = "./Inputs/coastal_polygons.geojson"

In [ ]:
import json
import ogr
import osr
import datacube
import matplotlib.pyplot as plt

from dea_datahandling import load_ard
from dea_datahandling import mostcommon_crs
from dea_bandindices import calculate_indices
from dea_coastaltools import tidal_tag
from dea_spatialtools import contour_extract
from dea_plotting import display_map
from dea_plotting import rgb
from dea_plotting import map_shapefile

dc = datacube.Datacube(app='erosion_contours')

with open(jsonfile) as f:
    data = json.load(f)

In [ ]:
# for feature in data['features']:
#     print(feature['geometry']['type'])
#     print(feature['geometry']['coordinates'])

feature = data['features'][0]
print(feature)

In [ ]:
polygon = ogr.CreateGeometryFromJson(str(feature['geometry']))

source = osr.SpatialReference()
source.ImportFromEPSG(4326)

target = osr.SpatialReference()
target.ImportFromEPSG(3577)

transform = osr.CoordinateTransformation(source, target)
polygon.Transform(transform)

print(polygon)

In [ ]:
poly_envelope = polygon.GetEnvelope()
minX, maxX, minY, maxY = poly_envelope

In [ ]:
# Get dc.load params
x_range = (minX, maxX)
y_range = (minY, maxY)
time_range = ('2000', '2018')
crs = "EPSG:3577"
res = (-30, 30)
tide_range = (0.50, 1.00)

products = ['ga_ls5t_ard_3', 'ga_ls7e_ard_3', 'ga_ls8c_ard_3']
measurements = ['nbart_red', 'nbart_green', 'nbart_blue', 'nbart_swir_1']

In [ ]:
query = {
    'y': y_range,
    'x': x_range,
    'time': time_range,
    'measurements': measurements,
    'crs': crs,
    'output_crs': crs,
    'resolution': res
}

landsat_ds = load_ard(
    dc=dc,
    products=products,
    group_by='solar_day',
    **query
)


In [ ]:
# Set the timesteps to visualise
timestep = 4

# Generate RGB plots at each timestep
rgb(landsat_ds, 
    index=timestep,
    percentile_stretch=(0.00, 0.95))


In [ ]:
landsat_ds = calculate_indices(landsat_ds, index='MNDWI', 
                               collection='ga_ls_3')

In [ ]:
# Calculate tides for each timestep in the satellite dataset
landsat_ds = tidal_tag(ds=landsat_ds, tidepost_lat=None, tidepost_lon=None)

# Print the output dataset with new `tide_height` variable
print(landsat_ds)

In [ ]:
# Calculate the min and max tide heights to include based on the % range
min_tide, max_tide = landsat_ds.tide_height.quantile(tide_range) 

# Plot the resulting tide heights for each Landsat image:
landsat_ds.tide_height.plot()
plt.axhline(min_tide, c='red', linestyle='--')
plt.axhline(max_tide, c='red', linestyle='--')
plt.show()


In [ ]:
# Keep timesteps larger than the min tide, and smaller than the max tide
landsat_filtered = landsat_ds.sel(time=(landsat_ds.tide_height > min_tide) &
                                       (landsat_ds.tide_height <= max_tide))
print(landsat_filtered)

In [ ]:
time_step = '1Y'

# Combine into summary images by `time_step`
landsat_summaries = (landsat_filtered.MNDWI
                     .compute()
                     .resample(time=time_step, closed='left')
                     .median('time'))

# Plot the output summary images
landsat_summaries.plot(col='time',
                       cmap='RdBu',
                       col_wrap=4,
                       vmin=-0.8,
                       vmax=0.8)
plt.show()


In [ ]:
# Set up attributes to assign to each waterline
attribute_data = {'time': [str(i)[0:10] for i in landsat_summaries.time.values]}
attribute_dtypes = {'time': 'str'}

# Extract waterline contours for the '0' water index threshold:
contour_gdf = contour_extract(
    z_values=[0],
    ds_array=landsat_summaries,
    ds_crs=landsat_ds.crs,
    ds_affine=landsat_ds.geobox.transform,
    output_shp=f'output_waterlines.shp',
    attribute_data=attribute_data,
    attribute_dtypes=attribute_dtypes,
    min_vertices=50)

# Plot output shapefile over the first MNDWI layer in the time series
landsat_summaries.isel(time=0).plot(size=12, 
                                    cmap='Greys', 
                                    add_colorbar=False)
contour_gdf.to_crs(epsg=4326).to_file('filename.geojson', driver='GeoJSON')

contour_gdf.plot(ax=plt.gca(), 
                 column='time', 
                 cmap='YlOrRd', 
                 legend=True, 
                 legend_kwds={'loc': 'lower right'})
plt.show()